In [50]:
import pandas as pd

In [51]:
data = pd.read_csv("orlen_stock.csv", sep=',')

In [52]:
data["Typ"] = ((data["Zamknięcie"] - data["Otwarcie"]) > 0).astype(int)

In [53]:
data

,Data,Otwarcie,Max,Min,Zamknięcie,Wolumen,Obrót,Typ
0,26.11.1999,22.20,22.40,21.40,22.10,10 902 704,477 538 435,0
1,29.11.1999,22.00,22.10,21.60,21.70,3 781 050,165 231 885,0
2,30.11.1999,21.90,22.30,21.90,22.10,2 260 083,99 895 669,1
3,01.12.1999,22.30,22.80,22.10,22.80,2 116 147,95 015 000,1
4,02.12.1999,23.00,23.70,22.90,23.20,2 271 721,105 862 199,1
...,...,...,...,...,...,...,...,...
6264,03.12.2024,51.91,52.00,51.06,51.33,1 995 684,102 505 672,0
6265,04.12.2024,51.58,52.15,51.23,51.66,1 506 019,77 799 504,1
6266,05.12.2024,51.80,52.75,51.31,52.44,1 493 160,77 852 496,1
6267,06.12.2024,52.50,53.15,52.00,52.14,1 446 088,76 014 952,0


In [54]:

"""
    Tutaj będziemy analizować, czy  formacja Młotek zachodzi
"""


class Pin_Low:
    def __init__(self, close, open, high, low):
        self.close = close
        self.open = open
        self.high = high
        self.low = low

    def conditions(self):
        try:
            if not all(isinstance(value, (int, float)) for value in [self.open, self.low, self.high, self.close]):
                raise ValueError("Wszystkie wartości (open, low, high, close) muszą być liczbami.")
        except(ValueError):
            print("Wszystkie wartości (open, low, high, close) muszą być liczbami.")
        condiction_1_1 = self.open - self.low
        condiction_1_2 = 0.6*(self.high - self.low)

        condiction_2 = self.close - self.open

        condiction_3_1 = self.high - self.close
        condiction_3_2 = 0.2*(self.high - self.low)

        return ((condiction_1_1 >= condiction_1_2) and
                (condiction_2 > 0) and
                (condiction_3_1 <= condiction_3_2))

In [55]:

"""
    Tutaj będziemy analizować, czy  formacja Młotek zachodzi
"""


class InvertedHammer:
    def __init__(self, close, open, high, low):
        self.close = close
        self.open = open
        self.high = high
        self.low = low

    def conditions(self):
        try:
            if not all(isinstance(value, (int, float)) for value in [self.open, self.low, self.high, self.close]):
                raise ValueError("Wszystkie wartości (open, low, high, close) muszą być liczbami.")
        except(ValueError):
            print("Wszystkie wartości (open, low, high, close) muszą być liczbami.")
        high_shadow = self.high - self.close
        low_shadow = self.open - self.low
        corpus = self.close - self.open
        length = self.high - self.low
        return (low_shadow <= 0.2*length) and (3*corpus > high_shadow > 2*corpus)
        

In [60]:
"""
    Ten skrypt posłuży nam, aby przeiterować przez dane, znaleźć pasuje do wzorca świece 
    
"""

def is_the_formation_successful(nr_candle: list):

    """
    Pokazuje czy formacja zadziałała prawidłowo
    :param nr_candle:
    :param close:
    :return: success_counter
    """
    success_counter = 0
    for nr in nr_candle:
        count = 0
        for num in range(5):
            if nr + num + 1 >= data.shape[0]:
                break
            candle = data.iloc[nr + num + 1].Typ
            if candle == 1:
                count += 1
        if count >= 3:
            success_counter += 1
    return success_counter


def counter():
    """
    zlicza ilość wystąpień formacji w danych oraz na której pozycji jest formacja i jej cene zamkniecia
    :return: counter, tab- tablica z indeksami formacji
    """
    counter = 0
    length = len(data)
    tab = []
    tab_close = []
    for i in range(length):
        close = data.iloc[i].Zamknięcie
        open = data.iloc[i].Otwarcie
        high = data.iloc[i].Max
        low = data.iloc[i].Min
        candle = InvertedHammer(close, open, high, low)
        flag = candle.conditions()
        if flag:
            counter += 1
            tab.append(i)
            tab_close.append(close)
    return counter, tab


results = counter()
print("Results:", results)

success_counter = is_the_formation_successful(results[1])
print("Success counter:", success_counter)

probability_of_success = (success_counter / results[0]) * 100
print("Probability:", probability_of_success, "%")

Results: (63, [4, 8, 252, 258, 469, 482, 501, 522, 669, 856, 868, 889, 997, 1082, 1088, 1102, 1443, 1451, 1513, 1581, 1587, 1601, 1694, 1712, 1719, 1991, 2001, 2248, 2254, 2335, 2736, 2751, 2793, 2803, 3095, 3105, 3268, 3336, 3394, 3396, 3404, 3595, 3765, 3909, 3971, 4123, 4366, 4383, 4526, 4787, 4882, 4925, 4926, 5042, 5110, 5273, 5590, 5633, 5672, 5916, 5969, 6095, 6113])
Success counter: 31
Probability: 49.2063492063492 %
